Create a Topic

In [4]:
from kafka.admin import KafkaAdminClient, NewTopic

def create_topics(list_topics):
    broker = "host.docker.internal:9092"
    admin = KafkaAdminClient(bootstrap_servers=broker)
    topics = [NewTopic(name=topic,num_partitions=1,replication_factor=1) for topic in list_topics]
    admin.create_topics(topics)

create_topics(["tpc-test-create"])


Consultando la descripción de los tópicos

In [3]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import UnknownTopicOrPartitionError


def describe_topics():
    broker = "host.docker.internal:9092"
    admin = KafkaAdminClient(bootstrap_servers=broker)
    topics_descriptions = admin.describe_topics(topics=["tpc-test"])
    return topics_descriptions

td = describe_topics()
td


[{'error_code': 0,
  'topic': 'tpc-test',
  'is_internal': False,
  'partitions': [{'error_code': 0,
    'partition': 0,
    'leader': 0,
    'replicas': [0],
    'isr': [0],
    'offline_replicas': []}]}]

Eliminando tópicos

In [5]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import UnknownTopicOrPartitionError

def delete_topics(list_topics:list):
    try:
        broker = "host.docker.internal:9092"
        admin = KafkaAdminClient(bootstrap_servers=broker)
        admin.delete_topics(list_topics)
    except Exception as ex:
        print(ex)

delete_topics(["tpc-test-create"])



Describiendo resource configs

-   El método describe_configs devuelve una lista de DescribeConfigsResponse_v2
-   Podemos acceder a los recursos con la propiedad resources

In [14]:
from kafka.admin import KafkaAdminClient, ConfigResource
from kafka.admin.config_resource import ConfigResourceType

TOPIC_NAME = "tpc-test"

def describe_configs():
    broker = "host.docker.internal:9092"
    admin = KafkaAdminClient(bootstrap_servers=broker)
    # 1. Define el recurso de configuración para el tópico
    config_resource = ConfigResource(ConfigResourceType.TOPIC, TOPIC_NAME)
    #Siempre que se buscar describir recursos se debe de pasarla una lista de instancias de ConfigResource
    configs = admin.describe_configs([config_resource])
    config_resources = [c.resources for c in configs]
    return config_resources

resources = describe_configs()
resources 


[[(0,
   '',
   2,
   'tpc-test',
   [('compression.type', 'producer', False, 5, False, []),
    ('confluent.tier.cleaner.compact.min.efficiency',
     '0.5',
     False,
     5,
     False,
     []),
    ('confluent.value.schema.validation', 'false', False, 5, False, []),
    ('leader.replication.throttled.replicas', '', False, 5, False, []),
    ('confluent.stray.log.max.deletions.per.run', '72', False, 5, False, []),
    ('confluent.key.subject.name.strategy',
     'io.confluent.kafka.serializers.subject.TopicNameStrategy',
     False,
     5,
     False,
     []),
    ('message.downconversion.enable', 'true', False, 5, False, []),
    ('min.insync.replicas', '1', False, 5, False, []),
    ('segment.jitter.ms', '0', False, 5, False, []),
    ('confluent.stray.log.delete.delay.ms', '604800000', False, 5, False, []),
    ('confluent.tier.cleaner.enable', 'false', False, 5, False, []),
    ('confluent.compacted.topic.prefer.tier.fetch.ms',
     '-1',
     False,
     5,
     False,
   

Exploring Consumer Groups

Listando todos los grupos

In [27]:
from kafka.admin import KafkaAdminClient

def list_consumer_groups():
    broker = "host.docker.internal:9092"
    admin = KafkaAdminClient(bootstrap_servers=broker)
    consumer_groups = admin.list_consumer_groups()
    return consumer_groups

groups = list_consumer_groups()

for g in groups:
    print(g)


('test_3', 'consumer')
('test_4', 'consumer')


-   Consultando la información de un grupo en específico
-   Retorna una lista de instancias de GroupInformation

In [24]:
from kafka.admin import KafkaAdminClient

def describe_consumer_group(consumer_group):
    broker = "host.docker.internal:9092"
    admin = KafkaAdminClient(bootstrap_servers=broker)
    consumer_groups = admin.describe_consumer_groups([consumer_group])
    return consumer_groups

groups = describe_consumer_group("test_2")

for g in groups:
    print(g)


GroupInformation(error_code=0, group='test_2', state='Dead', protocol_type='', protocol='', members=[], authorized_operations=None)


Listando el ultimo offset de un consumer_group

In [26]:
broker = "host.docker.internal:9092"
admin = KafkaAdminClient(bootstrap_servers=broker)
admin.list_consumer_group_offsets("test_3")
admin.list

{TopicPartition(topic='tpc-test', partition=0): OffsetAndMetadata(offset=3, metadata='')}

Creando mas particiones para un topico

In [36]:
from kafka.admin import KafkaAdminClient, NewPartitions

TOPIC_NAME = "tpc-test"
NUM_PARTITIONS = 5  # Asume que el número de particiones actual es 5. Cambia este valor según tu caso.

admin = KafkaAdminClient(bootstrap_servers="host.docker.internal:9092")

new_partitions = {TOPIC_NAME: NewPartitions(total_count=NUM_PARTITIONS + 2)}

admin.create_partitions(new_partitions)


CreatePartitionsResponse_v1(throttle_time_ms=0, topic_errors=[(topic='tpc-test', error_code=0, error_message=None)])

In [ ]:
from kafka import KafkaAdminClient, TopicPartition
from kafka.admin import DeleteRecordsOptions

admin = KafkaAdminClient(bootstrap_servers="your_bootstrap_servers")

# Aquí asumo que 'requestOlderOffsets' es un diccionario donde las claves son 
# instancias de TopicPartition y los valores son los offsets deseados.

older_offsets = admin.list_offsets(requestOlderOffsets)

records_to_delete = {tp: (0, offset) for tp, offset in older_offsets.items()}

options = DeleteRecordsOptions(timeout_ms=10000)  # Ajusta el timeout según lo necesario
admin.delete_records(records_to_delete, options)
